# 输入格式

"""
input : {"creative_id":"1,2,3,4,5",
        "ad_id":"1,2,3,4,5",
        "advertiser_id":"1,2,3,4,5",
        "product_id":"1,2,3,4,5",
        "product_category":"1,2,3,4,5",
        "industry":"1,2,3,4,5"}
        
label1 : age  sigmoid
label2 : gender softmax
"""

# 预训练embedding

# 加载数据
df_ad,df_click_log,df_user,df_click_log_test = loadData("./train/",mode="pro")

# 整合宽表
df_total_train = df_click_log.merge(df_user, how='left', on="user_id")
df_total_train = df_total_train.merge(df_ad, how='left', on="creative_id")
df_total_train = df_total_train.fillna("UNK").replace(["\\N"],"UNK")
df_total_train.head(5)

df_total_test = df_click_log_test.merge(df_ad, how='left', on="creative_id")
df_total_test = df_total_test.fillna("UNK").replace(["\\N"],"UNK")
df_total_test.head(5)

# 设定参数
features = ["creative_id","ad_id","product_id","product_category","advertiser_id","industry"]
SEQ_LEN = 45
negsample = 0
feature_max_idx = {}

dict_table = {}
max_num = {}



table = pickle.load(open("./"+method+"/data.table",'rb'))

for key in features:
    print("Starting to process " + key)
    df_total_train[key] = transform(df_total_train[key],table[key])
    df_total_test[key] = transform(df_total_test[key],table[key])
    
#pickle.dump(dict_table,open(base_path+"data.table",'wb'))

data_train = df_total_train.sort_values("time")
data_test = df_total_test.sort_values("time")

# 多线程生成序列

def fun_avg1(uId,groupData,val):
    return groupData[val].agg(lambda x : list(x))

def applyParallel(dfGrouped, func1,val):
    ret = Parallel(n_jobs=8)(delayed(func1)(name,group,val) for name, group in dfGrouped)
    return ret


# 多进程生成序列

def combine (pairs,key) : 
    
    return pairs[1][key].agg(lambda x : list(x))

combineAdd = partial(combine,key="creative_id")

def multiProcess (maxProcess = 5):
    res = []
    pool = Pool(maxProcess)
    res = pool.map(combineAdd,data.groupby("user_id",as_index=False))
    pool.close()
    pool.join()
    return res


#dataTrain = pd.read_csv("dataTrainStr.csv")
#dataTest = pd.read_csv("dataTestStr.csv")

pool = Pool(5)

df = {}

for item in features:
    
    print(item)
    
    combineAdd = partial(combine,key=item)
    
    df[item] = pool.map(combineAdd,dataTest.groupby("user_id",as_index=False))
    
pickle.dump(df,open("test_seq.table","wb"))

del df

pool.close()
pool.join()

In [1]:
from sklearn.preprocessing import LabelEncoder
from utils.utils import loadData,encode,transform,getSeq,getUserAvgSeq,getUserAvgSeq,upload
from model.W2V import wvmodel
from model.LGB import base_predict,base_train
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from utils.utils import encode,transform
import os
import tensorflow as tf
import pickle
from joblib import Parallel, delayed
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
from functools import partial

method = "test_nn_seq"

In [55]:
# 生成序列
features = {"creative_id":128,"ad_id":128,"product_id":32,"product_category":4,"advertiser_id":32,"industry":4}

#seq_train = pickle.load(open("train_seq.table","rb"))    ## no str

#seq_test = pickle.load(open("test_seq.table","rb"))      ## no str

#seq = {}

#for item in features :
    
    #seq[item] = seq_train[item] +  seq_test[item]

In [ ]:
for item,size in features.items() :

    print("Starting to train " + item)

    params = {"min_count":1,  # 保留的最小出现频次
          "alpha":0.1,       # 初始学习率
          "seed":2020,
          "min_alpha":0.01,   # 随着学习进行，学习率线性下降到这个最小数
          "window" : 5,   # 当前和预测单词之间的最大距离
          "size":size,     # 生成词向量的维度
          "compute_loss":False,# 损失(loss)值，如果是True 就会保存
          "workers":8}   # 几个CPU进行跑

    model = wvmodel(seq[item],10,"./test_nn_seq/" + item + ".w2v",params)

Starting to train creative_id
**********Word2Vector*********
**********initing*********
**********training*********
**********saving*********
Starting to train ad_id
**********Word2Vector*********
**********initing*********
